## Preparació

In [157]:
# Paquets

import json, os, urllib, zipfile
import pandas as pd
import geopandas as gpd
import xml.etree.ElementTree as ET

In [158]:
# Paràmetres

# Directori de treball
WORK_DIR = os.path.abspath(os.getcwd())

# Enllaços dades
URL_PUNTS_ANC = 'https://opendata-ajuntament.barcelona.cat/data/dataset/6778b87d-da09-4eb8-9504-5c9ae5fbfad9/resource/6cea1468-f18e-438b-b1c5-ea7c0e287412/download'
URL_APARCAMENT = 'https://opendata-ajuntament.barcelona.cat/data/dataset/a7fd022c-d703-49e3-b628-7b32599a6a54/resource/85457026-c48d-4a95-ac9e-080756375d50/download'
URL_SERVEIS = 'https://opendata-ajuntament.barcelona.cat/data/dataset/12bb6230-1a79-4371-89c5-d59aaa67152e/resource/f7e37859-7f3c-4dcb-ac70-91eae481dd26/download'

URL_BICING_INFO = 'https://api.bsmsa.eu/ext/api/bsm/gbfs/v2/en/station_information'
URL_BICING_STATUS = 'https://api.bsmsa.eu/ext/api/bsm/gbfs/v2/en/station_status'
#URL_BICING = 'https://www.bicing.barcelona/get-stations'


In [159]:
# Funcions

def get_data(url, filename):
    """Descarrega les dades a la carpeta 'datasets'."""
    try:
        urllib.request.urlretrieve(url, WORK_DIR + '/datasets/' + filename)
        print(filename + " descarregat correctament.")
    except:
        print("Problema descarregant: " + filename)
        
def save_data(gdf, filename):
    """Guarda un geodataframe en format SHP, el comprimeix i el desa a la carpeta 'map'."""
    gdf.to_file('tmp/' + filename + '.shp')
    zf = zipfile.ZipFile('map/' + filename + ".zip", "w", zipfile.ZIP_DEFLATED)
    zf.write('tmp/' + filename + ".dbf")
    zf.write('tmp/' + filename + ".shp")
    zf.write('tmp/' + filename + ".prj")
    zf.write('tmp/' + filename + ".shx")
    zf.write('tmp/' + filename + ".cpg")
    zf.close()
    print(filename + ' guardat correctament.')

## Punts d'ancoratge

In [160]:
get_data(URL_PUNTS_ANC, 'ancoratge.xml')
tree = ET.parse('datasets/ancoratge.xml')
root = tree.getroot()

ancoratge.xml descarregat correctament.


In [161]:
rows = []
for row in root.findall('.//row'):
    item = row.find("item")
    
    # Address
    address = ''
    if ET.iselement(item.find("address")):
        address = item.find("address").text
    
    # Coords
    lat = item.find("gmapx").text
    lon = item.find("gmapy").text
        
    rows.append({'address':address, 'lat':lat, 'lon':lon})

punts_ancoratge = pd.DataFrame(rows)

In [162]:
punts_ancoratge.head()

,address,lat,lon
0,C Torrent de l'Olla 183,41.405483,2.1533217
1,C Riera de Sant Miquel 49,41.398388,2.1556406
2,C Vic 25,41.399277,2.1547012
3,Trav Gràcia 115,41.399246,2.154404
4,C Luis Antúnez 1,41.39898,2.1562505


In [163]:
# Creació geodataframe
punts_ancoratge = gpd.GeoDataFrame(punts_ancoratge,
                                   geometry = gpd.points_from_xy(punts_ancoratge.lon, punts_ancoratge.lat))

# Conversió del sistema de referència espacial
punts_ancoratge = punts_ancoratge.set_crs('EPSG:4326')
punts_ancoratge.to_crs('EPSG:25831')

# Exportació dades
save_data(punts_ancoratge, 'PUNTS_ANCORATGE')

PUNTS_ANCORATGE guardat correctament.


## Serveis

In [164]:
# Captura i parsejament de dades

get_data(URL_SERVEIS, 'serveis.xml')
tree = ET.parse('datasets/serveis.xml')
root = tree.getroot()

rows = []
for row in root.findall('.//row'):
    item = row.find("item")
    
    # Name
    name = item.find("name").text
    
    # Address
    address = ''
    if ET.iselement(item.find("address")):
        address = item.find("address").text
    
    # Coords
    lat = item.find("gmapx").text
    lon = item.find("gmapy").text
    
    # Web
    web = ''
    if ET.iselement(item.find("code_url")):
        web = item.find("code_url").text

    # Phone
    phone = ''
    if ET.iselement(item.find("phonenumber")):
        phone = item.find("phonenumber").text
        
    rows.append({'name':name, 'address':address, 'lat':lat, 'lon':lon, 'web': web, 'phone': phone})

serveis = pd.DataFrame(rows)

serveis.xml descarregat correctament.


In [165]:
serveis.head()

,name,address,lat,lon,web,phone
0,Pura Vida Rickshaw Tours,C Lafont 19,41.372757,2.1689,www.puravida.barcelona,931823930
1,Roda Lliure,C Consell de Cent 595,41.405415,2.184575,,673 475 560
2,BiciHub *Can Picó,C Pere IV 58,41.395824,2.1921685,bicihub.barcelona,685 86 50 56
3,Barcelona Cycle Co,C Vigatans 2,41.384026,2.1796567,,933 153 259
4,Bikes Barcelona rental,C Marina 138,41.397278,2.1840746,www.bikesbarcelonarental.es,674 60 06 74


In [166]:
# Creació geodataframe
serveis = gpd.GeoDataFrame(serveis, geometry = gpd.points_from_xy(serveis.lon, serveis.lat))

# Conversió del sistema de referència espacial
serveis = serveis.set_crs('EPSG:4326')
serveis.to_crs('EPSG:25831')

# Exportació dades
save_data(serveis, 'SERVEIS')

SERVEIS guardat correctament.


## Bicing

In [142]:
# Captura dades

# Informació de les estacions
with urllib.request.urlopen(URL_BICING_INFO) as url:
    data = json.loads(url.read().decode())

bicing_info = pd.DataFrame(data['data']['stations'])

# Informació de l'estat de les estacions
with urllib.request.urlopen(URL_BICING_STATUS) as url:
    data = json.loads(url.read().decode())

bicing_status = pd.DataFrame(data['data']['stations'])

In [143]:
# Unim dades d'informació i estat
bicing_stations = pd.merge(bicing_info, bicing_status, on='station_id')

# Seleccionem columnes
COLUMNS = ['station_id','name','lat','lon','altitude','address','capacity','num_bikes_available','status']
bicing_stations = bicing_stations[COLUMNS]

# Generem un geodataframe
bicing = gpd.GeoDataFrame(bicing_stations, geometry = gpd.points_from_xy(bicing_stations.lon, bicing_stations.lat))

# Conversió del sistema de referència espacial
bicing = bicing.set_crs('EPSG:4326')
bicing.to_crs('EPSG:25831')

bicing.head()

,station_id,name,lat,lon,altitude,address,capacity,num_bikes_available,status,geometry
0,1,"GRAN VIA CORTS CATALANES, 760",41.397978,2.180107,16.0,"GRAN VIA CORTS CATALANES, 760",46,8,IN_SERVICE,POINT (2.18011 41.39798)
1,2,"C/ ROGER DE FLOR, 126",41.395488,2.177198,17.0,"C/ ROGER DE FLOR, 126",27,2,IN_SERVICE,POINT (2.17720 41.39549)
2,3,"C/ NÀPOLS, 82",41.394156,2.181331,11.0,"C/ NÀPOLS, 82",27,10,IN_SERVICE,POINT (2.18133 41.39416)
3,4,"C/ RIBES, 13",41.393317,2.181248,8.0,"C/ RIBES, 13",21,2,IN_SERVICE,POINT (2.18125 41.39332)
4,5,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",41.391103,2.180176,7.0,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",39,5,IN_SERVICE,POINT (2.18018 41.39110)


In [25]:
save_data(bicing, 'BICING')

BICING guardat correctament.
